In [171]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%matplotlib inline

from pathlib import Path

In [411]:
from attila.util.config import get_env
from attila.util.plots import plot_history, plot_preds
from attila.util.io import load_pickle, append_rows2text, load_json, get_summary, dirs, stuff2pickle
from attila.experiments.tools import runs2tex

In [7]:
_here = Path('.').resolve()

In [9]:
config, data_path, out_path, models_config_path = get_env(_here)

In [182]:
config.set('training', 'batch size', '4')
config.set('training', 'epochs', '50')
config.set('data', 'aug', 'False')

In [ ]:
get_figa(1, 1)

metrics = [
    {
        'name': 'attila_metrics_mean_IoU',
        'callback': mean_IoU(get_batch_mean=False)
    },
    {
        'name': 'attila_metrics_DSC',
        'callback': DSC(get_batch_mean=False)
    }
]

results = {
    m: {
        'mean': [],
        'std': []
    }
    for m in [ x['name'] for x in metrics]
}
ratios = []

for folder in dirs(out_path / 'trials' / 'to-aug-or-not'):
    tt_ratio = np.float32(str(Path(folder).name).split('-')[-1])
    ratios.append(tt_ratio)
    summary = get_summary(folder, config)

    y_true_batch = summary['preds'][1]
    y_pred_batch = summary['preds'][2]

    for m in metrics:
        key = m['name']
        f = m['callback']

        vals = f(
            normalize_transformation((0, 1))(y_true_batch),
            normalize_transformation((0, 1))(y_pred_batch)
        ).numpy()

        results[key]['mean'].append(vals.mean())
        results[key]['std'].append(vals.std())

for m in metrics:
    key = m['name']
    plt.errorbar(
        ratios,
        results[key]['mean'],
        yerr=results[key]['std'],
        fmt='o',
        label=key
    )
    plt.plot(ratios[np.argmax(results[key]['mean'])], np.max(results[key]['mean']), marker='x', color='r', markersize=12)

plt.xlabel('validation set size (out of 18 training images)')
plt.title('Vanilla U-Net')
plt.legend()

# stuff2pickle(results, 'to-aug-or-not.pkl')

In [413]:
learning_rates

[1e-06,
 1.6238e-06,
 2.6367e-06,
 4.2813e-06,
 6.9519e-06,
 1.12884e-05,
 1.83298e-05,
 2.97635e-05,
 4.83293e-05,
 7.8476e-05,
 0.0001274275,
 0.0002069138,
 0.0003359818,
 0.0005455595,
 0.0008858668,
 0.00143845,
 0.0023357214,
 0.00379269,
 0.0061584823,
 0.01]

In [206]:
plot_preds(
    summary['preds'],
    config.get('image', 'cmap'),
    out_folder=model_folder
)

pred img saved in /home/stefano/_tmp/_martin/pred_0.png
pred img saved in /home/stefano/_tmp/_martin/pred_1.png
pred img saved in /home/stefano/_tmp/_martin/pred_2.png
pred img saved in /home/stefano/_tmp/_martin/pred_3.png
pred img saved in /home/stefano/_tmp/_martin/pred_4.png
pred img saved in /home/stefano/_tmp/_martin/pred_5.png
pred img saved in /home/stefano/_tmp/_martin/pred_6.png
pred img saved in /home/stefano/_tmp/_martin/pred_7.png
